In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

import mmdet
print(mmdet.__version__)

from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# Choose to use a config and initialize the detector
config = 'mmdetection/configs/hrnet/cascade_rcnn_hrnetv2p_w32_20e_coco.py'
# Setup a checkpoint file to load
checkpoint = 'mmdetection/checkpoints/cascade_rcnn_hrnetv2p_w32_20e_coco_20200208-928455a4.pth'
# initialize the detector
model = init_detector(config, checkpoint, device='cuda:0')

In [ ]:
img = 'mmdetection/demo/demo-person.png'
result = inference_detector(model, img)

In [ ]:
show_result_pyplot(model, img, result, score_thr=0.3)

In [ ]:
import os
import cv2
import re
import glob
import time
import json
import mmcv
import shutil
import random
import argparse
import os.path as osp
import numpy as np
import pycocotools.mask as maskUtils
import xml.etree.ElementTree as ET
import cityscapesscripts.helpers.labels as CSLabels
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from typing import Dict, List
from PIL import Image
from collections import defaultdict, OrderedDict
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

In [ ]:
!pip install cityscapesscripts --index-url https://pypi.douban.com/simple 
!pip install pycocotools --index-url https://pypi.douban.com/simple 

In [ ]:
%%writefile labels.txt
pedestrian

In [ ]:
from mmcv import Config
cfg = Config.fromfile('mmdetection/configs/hrnet/cascade_rcnn_hrnetv2p_w32_20e_coco.py')

In [ ]:
from mmdet.apis import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'CocoDataset'
cfg.classes = 'labels.txt'
cfg.data_root = 'citypersons'
# cfg.model.bbox_head.num_classes = 1
cfg.model.roi_head.bbox_head[0].num_classes = 1
cfg.model.roi_head.bbox_head[1].num_classes = 1
cfg.model.roi_head.bbox_head[2].num_classes = 1
cfg.data.test.type = 'CocoDataset'
cfg.data.test.classes = 'labels.txt'
cfg.data.test.data_root = 'citypersons'
cfg.data.test.ann_file = 'val_gt.json'
cfg.data.test.img_prefix = 'val'
cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = 'citypersons'
cfg.data.train.ann_file = 'train_gt.json'
cfg.data.train.img_prefix = 'train'
cfg.data.train.classes = 'labels.txt'
cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = 'citypersons'
cfg.data.val.ann_file = 'val_gt.json'
cfg.data.val.img_prefix = 'val'
cfg.data.val.classes = 'labels.txt'

# modify num classes of the model in box head
# cfg.model.roi_head.bbox_head.num_classes = 1
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
cfg.load_from = 'mmdetection/checkpoints/cascade_rcnn_hrnetv2p_w32_20e_coco_20200208-928455a4.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.01 / 8
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
# cfg.lr_config.warmup = None
# cfg.lr_config.policy = 'cyclic'
# cfg.log_config.interval = 10
cfg.lr_config.policy = 'step'
cfg.lr_config.step = 7
cfg.data.samples_per_gpu = 1
cfg.data.workers_per_gpu = 1
cfg.evaluation.metric = 'bbox'
cfg.evaluation.interval = 1
cfg.checkpoint_config.interval = 1
cfg.runner.max_epochs = 1
cfg.log_config.interval = 100

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device = 'cuda'

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
datasets = [build_dataset(cfg.data.train)]
model = build_detector(
 cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES

mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
results = []
model = init_detector(cfg, 'epoch_1.pth')
items = os.listdir("citypersons/val/")
for item in items:
    img = mmcv.imread("citypersons/val/"+item)
    result = inference_detector(model, img)
    results.append(result)
    show_result_pyplot(model, img, result)

In [ ]:
START_BOUNDING_BOX_ID = 1
PRE_DEFINE_CATEGORIES = {}
categories = PRE_DEFINE_CATEGORIES
width = int(2048)
height = int(1024)
bnd_id = START_BOUNDING_BOX_ID
json_dict = {"images":[], "type": "instances", "annotations": [],
            "categories": []}
def get_filename_as_int(filename):
    try:
        filename = os.path.splitext(filename)[0]
        filename = filename.split('_')[1] + filename.split('_')[2]
        return int(filename)
    except:
        raise NotImplementedError('Filename %s is supposed to be an integer.'%(filename))

results = []
model = init_detector(cfg, 'mmdetection/checkpoints/epoch_72.pth')
items = os.listdir("citypersons/val/")
ids = 1
for item in items:
    img = mmcv.imread("citypersons/val/"+item)
    result = inference_detector(model, img)          
    image_id = get_filename_as_int(item)
    image = {'file_name': item, 'height': height, 'width': width,
             'id': ids}
    json_dict['images'].append(image)
    for i in result[0]:
        xmin = int(float(i[0]))
        ymin = int(float(i[1]))
        xmax = int(float(i[2]))
        ymax = int(float(i[3]))
        scores = float(i[4])
        o_width = xmax-xmin
        o_height = ymax-ymin
        bbox = [xmin,ymin,o_width,o_height] 
        ann = {'area': o_width*o_height, 'iscrowd': 0, 'image_id':
                ids, 'bbox':bbox,
                'category_id': 1, 'id': bnd_id, 'ignore': 0,
                'segmentation': [], 'height': o_height, 'score': scores}
        json_dict['annotations'].append(ann)
        bnd_id = bnd_id + 1
    ids += 1
with open('val_dt_cascade72.json', 'w') as json_fp:
    json_str1 = json.dumps(json_dict)
    json_fp.write(json_str1)
    
with open('resFile_cascade72.json', 'w') as json_res:
    json_str2 = json.dumps(json_dict['annotations'])
    json_res.write(json_str2)

In [ ]:
from coco import COCO
from eval_MR_multisetup import COCOeval

annType = 'bbox'      #specify type here
print ('Running demo for *%s* results.'%(annType))

#initialize COCO ground truth api
annFile = 'val_gt.json'
# initialize COCO detections api
resFile = 'resFile_cascade72.json'

## running evaluation
res_file = open("results-cascade-72epochs.txt", "w")
for id_setup in range(0,4):
    cocoGt = COCO(annFile)
    cocoDt = cocoGt.loadRes(resFile)
    imgIds = sorted(cocoGt.getImgIds())
    cocoEval = COCOeval(cocoGt,cocoDt,annType)
    cocoEval.params.imgIds  = imgIds
    cocoEval.evaluate(id_setup)
    cocoEval.accumulate()
    cocoEval.summarize(id_setup,res_file)

res_file.close()